In [40]:
import cv2
import numpy as np
import os         
from random import shuffle 
from tqdm import tqdm      
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [41]:
%matplotlib inline
TRAIN_DIR = 'train'
IMG_SIZE = 100
LR = 1e-3
MODEL_NAME = 'dogs-vs-cats'

In [42]:
def create_label(image_name):
   
    word_label = image_name.split('.')[-3]
    if word_label == 'cat':
        return np.array([1,0])
    elif word_label == 'dog':
        return np.array([0,1])

In [43]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data), create_label(img)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [44]:
train_data= create_train_data()

100%|█████████████████████████████████████| 1715/1715 [00:03<00:00, 467.87it/s]


In [45]:
train=train_data[:-100]
test=train_data[-100:]

In [46]:
X_train = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = [i[1] for i in test]

In [47]:
from keras.utils import to_categorical
train_Y_one_hot = to_categorical(y_train)
print(X_train.shape)
train_Y_one_hot.shape
#Y_test = to_categorical(Y_test)



(1615, 100, 100, 1)


(1615, 2, 2)

In [48]:
tf.reset_default_graph()
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
print(convnet)

convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
model = tflearn.DNN(convnet, tensorboard_dir='log', tensorboard_verbose=0)
"""model.fit({'input': X_train}, {'targets': y_train}, n_epoch=10, 
          validation_set=({'input': X_test}, {'targets': y_test}), 
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME) """

Tensor("Conv2D/Relu:0", shape=(?, 100, 100, 32), dtype=float32)


"model.fit({'input': X_train}, {'targets': y_train}, n_epoch=10, \n          validation_set=({'input': X_test}, {'targets': y_test}), \n          snapshot_step=500, show_metric=True, run_id=MODEL_NAME) "

In [49]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [50]:
batch_size = 64
epochs = 10
num_classes = 2


In [51]:
X_train.shape

(1615, 100, 100, 1)

In [52]:
for x in range(0,len(y_train)):
    if(y_train[x][0]==1):
        y_train[x]=1;
    else:
        y_train[x]=0;
for x in range(0,len(y_test)):
    if(y_test[x][0]==1):
        y_test[x]=1;
    else:
        y_test[x]=0;
        



In [53]:
y_train = to_categorical(y_train)
y_test=to_categorical(y_test)
y_train.shape,X_train.shape,y_test.shape,X_test.shape

((1615, 2), (1615, 100, 100, 1), (100, 2), (100, 100, 100, 1))

In [57]:
keras.backend.clear_session()
from keras.layers import Activation, Dropout, Flatten, Dense

"""binary_model = Sequential()
binary_model.add(Conv2D(16, kernel_size=(32, 5),activation='linear',input_shape=(50,50,1),padding='same'))
binary_model.add(LeakyReLU(alpha=0.1))
binary_model.add(MaxPooling2D((3, 5),padding='same'))
binary_model.add(Dropout(0.2))
binary_model.add(Conv2D(32, (64, 5), activation='linear',padding='same'))
binary_model.add(LeakyReLU(alpha=0.1))
binary_model.add(MaxPooling2D(pool_size=(3, 5),padding='same'))
binary_model.add(Dropout(0.2))
binary_model.add(Flatten())
binary_model.add(Dense(128, activation='linear'))
binary_model.add(LeakyReLU(alpha=0.1))                  
binary_model.add(Dense(num_classes, activation='softmax'))"""
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [58]:

#binary_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [59]:
binary_train = model.fit(X_train, y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test, y_test))

Train on 1615 samples, validate on 100 samples
Epoch 1/10
1615/1615 [==============================] - ETA: 38s - loss: 4.5590 - acc: 0.50 - ETA: 29s - loss: 5.2867 - acc: 0.56 - ETA: 26s - loss: 5.7811 - acc: 0.56 - ETA: 24s - loss: 5.6508 - acc: 0.59 - ETA: 22s - loss: 6.2239 - acc: 0.56 - ETA: 21s - loss: 6.4389 - acc: 0.56 - ETA: 19s - loss: 6.4494 - acc: 0.56 - ETA: 18s - loss: 6.3635 - acc: 0.57 - ETA: 17s - loss: 6.4357 - acc: 0.57 - ETA: 16s - loss: 6.6187 - acc: 0.56 - ETA: 15s - loss: 6.4496 - acc: 0.57 - ETA: 14s - loss: 6.5488 - acc: 0.57 - ETA: 13s - loss: 6.6231 - acc: 0.56 - ETA: 11s - loss: 6.5436 - acc: 0.57 - ETA: 10s - loss: 6.4747 - acc: 0.58 - ETA: 9s - loss: 6.4301 - acc: 0.5850 - ETA: 8s - loss: 6.4791 - acc: 0.582 - ETA: 7s - loss: 6.5505 - acc: 0.579 - ETA: 6s - loss: 6.5814 - acc: 0.577 - ETA: 5s - loss: 6.5905 - acc: 0.577 - ETA: 4s - loss: 6.6226 - acc: 0.576 - ETA: 3s - loss: 6.6403 - acc: 0.575 - ETA: 2s - loss: 6.5857 - acc: 0.579 - ETA: 1s - loss: 6.6348

1615/1615 [==============================] - ETA: 28s - loss: 0.7983 - acc: 0.58 - ETA: 27s - loss: 1.8284 - acc: 0.58 - ETA: 26s - loss: 1.6283 - acc: 0.56 - ETA: 25s - loss: 1.3848 - acc: 0.57 - ETA: 24s - loss: 1.2408 - acc: 0.57 - ETA: 22s - loss: 1.1447 - acc: 0.57 - ETA: 21s - loss: 1.0668 - acc: 0.59 - ETA: 20s - loss: 1.0168 - acc: 0.59 - ETA: 19s - loss: 0.9760 - acc: 0.59 - ETA: 18s - loss: 0.9397 - acc: 0.60 - ETA: 17s - loss: 0.9175 - acc: 0.60 - ETA: 16s - loss: 0.8918 - acc: 0.60 - ETA: 15s - loss: 0.8728 - acc: 0.61 - ETA: 13s - loss: 0.8562 - acc: 0.61 - ETA: 12s - loss: 0.8413 - acc: 0.61 - ETA: 11s - loss: 0.8273 - acc: 0.61 - ETA: 10s - loss: 0.8153 - acc: 0.61 - ETA: 8s - loss: 0.8020 - acc: 0.6241 - ETA: 7s - loss: 0.8045 - acc: 0.622 - ETA: 6s - loss: 0.7957 - acc: 0.624 - ETA: 5s - loss: 0.7879 - acc: 0.625 - ETA: 3s - loss: 0.7800 - acc: 0.624 - ETA: 2s - loss: 0.7752 - acc: 0.625 - ETA: 1s - loss: 0.7700 - acc: 0.623 - ETA: 0s - loss: 0.7654 - acc: 0.622 - 31s 